In [1]:
import keras
import json
import numpy as np
from nltk import word_tokenize, sent_tokenize
from keras.models import Sequential, Model
from keras.layers import Bidirectional, LSTM, Convolution2D, Dense, Embedding, concatenate, dot, Merge, Input, multiply,Activation
from keras.optimizers import RMSprop
from keras.preprocessing.sequence import pad_sequences

Using Theano backend.


In [2]:
context_train = json.load(open('data/squad/context_train.json'))
query_train = json.load(open('data/squad/query_train.json'))


In [3]:
shared = json.load(open('data/squad/shared.json'))

In [4]:
word_emb_size = 100
T = len(context_train['context_idx'][0]) * len(context_train['context_idx'][0][0])
J = len(query_train['question_idx'][0])

In [5]:
T

11194

In [6]:
context_train.keys()

dict_keys(['actual_context', 'char_context', 'word_context', 'context_idx'])

In [9]:
query_train.keys()

dict_keys(['answer_end', 'question_idx', 'answer_start', 'word_question', 'relation', 'char_question'])

In [10]:
shared.keys()

dict_keys(['char_counter', 'word2vec', 'all_words', 'word_counter'])

In [11]:
each_context = []
for context in context_train['context_idx']:
    each_sent = []
    for sent in context:
        each_sent.extend(sent)  
    each_context.append(each_sent)    
        
        

In [12]:
len(each_context[0])

11194

In [13]:
emb_mat = np.array([shared['word2vec'][word] if word in shared['word2vec']
                        else np.random.multivariate_normal(np.zeros(word_emb_size), np.eye(word_emb_size))
                        for word in shared['all_words']])

In [14]:
emb_mat.shape

(95962, 100)

In [38]:
model1_input = Input(shape = (T,))
model1_embed = Embedding(output_dim=100, input_dim=emb_mat.shape[0], weights = [emb_mat], trainable = False)(model1_input)
model1_lstm = Bidirectional(LSTM(256, return_sequences = True))(model1_embed)

In [39]:
from keras import backend as K

In [40]:
model2_input = Input(shape = (J, ))
model2_embed = Embedding(output_dim=100, input_dim=emb_mat.shape[0], weights = [emb_mat], trainable = False)(model2_input)
model2_lstm = Bidirectional(LSTM(256, return_sequences = True))(model2_embed)

In [50]:
multiply_layer = dot([model1_lstm,model2_lstm], axes=2)

In [42]:
# modelling_model1 = Sequential()
modelling_bilstm_layer1 = Bidirectional(LSTM(256, return_sequences = True))(multiply_layer)
modelling_bilstm_layer2 = Bidirectional(LSTM(256, return_sequences = True))(modelling_bilstm_layer1)                    

In [43]:
output_start_index = Dense(T, activation='softmax')(modelling_bilstm_layer2)

In [44]:
merge_layer = concatenate([modelling_bilstm_layer2, output_start_index])
print(merge_layer.shape)

Shape.0


In [45]:
output_end_index_lstm = LSTM(256)(merge_layer)
output_end_index_dense = Activation('softmax')(output_end_index_lstm)

In [46]:
final_model = Model(inputs=[model1_input, model2_input], outputs=[output_start_index, output_end_index_dense])

In [47]:
final_model.compile(optimizer='rmsprop', loss='categorical_crossentropy')

In [48]:
final_model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_3 (InputLayer)             (None, 11194)         0                                            
____________________________________________________________________________________________________
input_4 (InputLayer)             (None, 60)            0                                            
____________________________________________________________________________________________________
embedding_3 (Embedding)          (None, 11194, 100)    9596200     input_3[0][0]                    
____________________________________________________________________________________________________
embedding_4 (Embedding)          (None, 60, 100)       9596200     input_4[0][0]                    
___________________________________________________________________________________________

In [58]:
final_model.fit(
    [np.array(each_context[:20000]), np.array(query_train['question_idx'][:20000])],
    [np.array(query_train['answer_start'][:20000]), np.array(query_train['answer_end'][:20000])],
    batch_size=60,
    validation_split = 0.2
)

ValueError: Error when checking target: expected dense_2 to have shape (None, 11194, 11194) but got array with shape (20000, 1, 2)

In [52]:
np.array(each_context).shape

(20963, 11194)

In [54]:
np.array(query_train['question_idx']).shape

(98169, 60)

In [37]:
len(context_train['word_context'])

20963

In [77]:
np.array([[2,3, [5]], [4, 5, [9]]]).flatten()

ValueError: setting an array element with a sequence.

In [56]:
len(context_train['context_idx'])

20963

In [57]:
len(query_train['question_idx'])

98169